In [1]:
######### Takes in the raw data game data just after the scrape and creates yearly summary stats for team


import pandas as pd
import numpy as np
import os
import ast # for literal_eval function



In [2]:

# Load Data
conference_df = pd.read_csv('../data/yearly_conference_members.csv')
games_df = pd.read_csv('../data/cfb_scrape_raw.csv')

# rename Unnamed: 0 to year in conference_df
conference_df = conference_df.rename(columns={'Unnamed: 0': 'Year'})



# Convert string representations of lists back into actual lists
for col in conference_df.columns[1:]:
    conference_df[col] = conference_df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


C:\Users\Justin\AppData\Local\Temp\ipykernel_2484\4288072279.py:3: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  games_df = pd.read_csv('../data/cfb_scrape_raw.csv')


In [3]:
### Extract the ranking from the team name where applicable
## Pool ranking is contained in parentheses at the beginning of the team name string of the Winner and Loser columns
def extract_ranking(team_name):
    """Extract the ranking from the team name where applicable."""
    # If the team name does not start with a parenthesis, there is no ranking
    if team_name[0] != '(':
        return None
    
    # Find the closing parenthesis
    closing_paren = team_name.find(')')
    
    # Extract the ranking
    ranking = team_name[1:closing_paren]
    
    return ranking

# Apply function to Winner column and save in new 'Winner_Rank' column
games_df['Winner_Rank'] = games_df['Winner'].apply(extract_ranking)
# Apply function to Loser column and save in new 'Loser_Rank' column
games_df['Loser_Rank'] = games_df['Loser'].apply(extract_ranking)

# Remove the ranking from the team name but only from the front of the string
games_df['Winner'] = games_df['Winner'].str.replace(r'^\(\d+\)\s', '')
games_df['Loser'] = games_df['Loser'].str.replace(r'^\(\d+\)\s', '')

# Strip leading and trailing whitespace from team names
games_df['Winner'] = games_df['Winner'].str.strip()
games_df['Loser'] = games_df['Loser'].str.strip()

C:\Users\Justin\AppData\Local\Temp\ipykernel_2484\991721257.py:23: FutureWarning: The default value of regex will change from True to False in a future version.
  games_df['Winner'] = games_df['Winner'].str.replace(r'^\(\d+\)\s', '')
C:\Users\Justin\AppData\Local\Temp\ipykernel_2484\991721257.py:24: FutureWarning: The default value of regex will change from True to False in a future version.
  games_df['Loser'] = games_df['Loser'].str.replace(r'^\(\d+\)\s', '')


In [4]:
conference_df = pd.read_csv('../data/yearly_conference_members.csv')

# rename Unnamed: 0 to year in conference_df
conference_df = conference_df.rename(columns={'Unnamed: 0': 'Year'})

# Convert string representations of lists back into actual lists
for col in conference_df.columns[1:]:
    conference_df[col] = conference_df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


# Function to retrieve the conference based on year and team name
def get_conference(year, team_name, conference_lookup):
    """
    Given a year and team name, retrieve the conference the team belongs to.
    If no conference found, return "Independent/Unknown".
    """
    return conference_lookup.get((year, team_name), "Independent/Unknown")

# Create a lookup dictionary for team-year to conference mapping
conference_lookup = {}

for _, row in conference_df.iterrows():
    year = row['Year']
    for col, teams in row[1:].items():
        for team in teams:
            conference_lookup[(year, team)] = col

# Use funtion to assign conference affiliation the teams in each row but only if 
# the team name is not null
games_df['winner_conference'] = games_df.apply(lambda row: get_conference(row['Season'], row['Winner'], conference_lookup) if pd.notna(row['Winner']) else None, axis=1)
games_df['loser_conference'] = games_df.apply(lambda row: get_conference(row['Season'], row['Loser'], conference_lookup) if pd.notna(row['Loser']) else None, axis=1)


### Create a new column for conference and non-conference games
# Function to determine if a game is a conference game
def determine_conference_game(conference1, conference2):
    if conference1 == conference2:
        return 'Conference'
    else:
        return 'Non-Conference'
    
# Apply to create new 'game_type' column using the determine_conference_game function and the winner and loser conference columns
games_df['game_type'] = games_df.apply(lambda row: determine_conference_game(row['winner_conference'], row['loser_conference']) if pd.notna(row['winner_conference']) and pd.notna(row['loser_conference']) else None, axis=1)

# Set game type to Non-Conferecne if either team is Independent/Unknown
games_df.loc[games_df['winner_conference'] == 'Independent/Unknown', 'game_type'] = 'Non-Conference'
games_df.loc[games_df['loser_conference'] == 'Independent/Unknown', 'game_type'] = 'Non-Conference'

# Rename the points columns to be more to match code down the line
# Pts = winner_pts, Pts.1 = loser_pts
games_df = games_df.rename(columns={'Pts': 'winner_pts', 'Pts.1': 'loser_pts'})

# season_year to season
games_df = games_df.rename(columns={'Season': 'season'})

# unnamed: 6 to loc_ind
games_df = games_df.rename(columns={'Unnamed: 6': 'loc_ind'})
# unameed: 7 to loc_ind_2
games_df = games_df.rename(columns={'Unnamed: 7': 'loc_ind_2'})
# combine loc_ind and loc_ind_2 to loc_ind
games_df['loc_ind'] = games_df['loc_ind'].fillna(games_df['loc_ind_2'])

# drop loc_ind_2
games_df = games_df.drop(columns=['loc_ind_2'])

# make all column names lowercase
games_df.columns = games_df.columns.str.lower()

In [5]:
## Classify games base on Notes column (creating seperate columns for conf_champ_games, bowl_games, playoff_games)

# Do The playoff first to make filtering out the conf_champ games easier
# Function to determine if a game is a playoff game
def determine_playoff_game(notes):
    if pd.notna(notes) and 'Playoff' in notes:
        return notes
    else:
        return ''
    
# run function
games_df['playoff_game'] = games_df['notes'].apply(determine_playoff_game)
# If the playoff_game column is not empty clear the notes column
games_df.loc[games_df['playoff_game'] != '', 'notes'] = ''

# Function to determine if a game is a conference championship game
def determine_conf_champ_game(notes):
    if pd.notna(notes) and 'Championship' in notes:
        return notes
    else:
        return ''
# run function
games_df['conf_champ_game'] = games_df['notes'].apply(determine_conf_champ_game)
# If the conf_champ_game column is not empty clear the notes column
games_df.loc[games_df['conf_champ_game'] != '', 'notes'] = ''

# Classify bowl games
# Function to determine if a game is a bowl game
def determine_bowl_game(notes):
    if pd.notna(notes) and 'Bowl' in notes:
        return notes
    else:
        return ''
    
# run function
games_df['bowl_game'] = games_df['notes'].apply(determine_bowl_game)
# If the bowl_game column is not empty clear the notes column
games_df.loc[games_df['bowl_game'] != '', 'notes'] = ''




In [6]:
len(games_df)
# output csv for check
# games_df.to_csv('../TEMP/check.csv', index=False)

81606

In [7]:
## add a filter to only include game that are teams in the ncaa_division_1_fbs dataframe

# load the ncaa_division_1_fbs dataframe
ncaa_df = pd.read_csv('../data/ncaa_com_fbs_fcs_table_with_location.csv')
ncaa_df.columns

# simple_name to a list
ncaa_names = ncaa_df['simple_name'].tolist()

# print length of list

print (len(ncaa_names))
print (len(games_df))

# filter the games_df to only include games from the relevant teams
# team name will be in winner or loser column
games_df = games_df[games_df['winner'].isin(ncaa_names) | games_df['loser'].isin(ncaa_names)]

print(len(games_df))

games_df.columns

261
81606
77718


Index(['rk', 'wk', 'date', 'day', 'winner', 'winner_pts', 'loc_ind', 'loser',
       'loser_pts', 'notes', 'season', 'time', 'winner_rank', 'loser_rank',
       'winner_conference', 'loser_conference', 'game_type', 'playoff_game',
       'conf_champ_game', 'bowl_game'],
      dtype='object')

In [8]:
# classify game as close or blowout based on the score


# absolute value is  less than 7 is close and greater than 21 is blowout
def classify_game(row):
    if abs(row['winner_pts'] - row['loser_pts']) < 7:
        return 'Close'
    elif abs(row['winner_pts'] - row['loser_pts']) > 21:
        return 'Blowout'
    else:
        return ''
    
# apply function to create new column

games_df['margin'] = games_df.apply(classify_game, axis=1)


games_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77718 entries, 0 to 81605
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rk                 77718 non-null  int64  
 1   wk                 77718 non-null  int64  
 2   date               77718 non-null  object 
 3   day                77718 non-null  object 
 4   winner             77718 non-null  object 
 5   winner_pts         77716 non-null  float64
 6   loc_ind            32109 non-null  object 
 7   loser              77718 non-null  object 
 8   loser_pts          77716 non-null  float64
 9   notes              5158 non-null   object 
 10  season             77718 non-null  int64  
 11  time               7343 non-null   object 
 12  winner_rank        14813 non-null  object 
 13  loser_rank         5880 non-null   object 
 14  winner_conference  77718 non-null  object 
 15  loser_conference   77718 non-null  object 
 16  game_type          777

In [9]:
# Create a new column for home Team and away team and move the tean names and score from winners and losers coulmn to 
## the new columns based on the loc_ind column - leaving the Nuetral games in the winners and losers columns

# Function to determine the home team and away team
def determine_home_away_team(loc_ind, winner, loser):
    if loc_ind == '@':
        return loser, winner
    elif loc_ind == 'N':
        return '', ''
    else:
        return winner, loser
    
# Apply function to create new columns for home and away team
games_df['home_team'], games_df['away_team'] = zip(*games_df.apply(lambda row: determine_home_away_team(row['loc_ind'], row['winner'], row['loser']), axis=1))

# Function to determine the home team score and away team score
def determine_home_away_score(loc_ind, winner_pts, loser_pts):
    if loc_ind == '@':
        return loser_pts, winner_pts
    elif loc_ind == 'N':   
        return '', ''
    else:
        return winner_pts, loser_pts

# Function to assign the ranking columns to new coumns (home_rank, away_rank) if the game was not nuetral
def determine_home_away_rank(loc_ind, winner_rank, loser_rank):
    if loc_ind == '@':
        return loser_rank, winner_rank
    elif loc_ind == 'N':
        return '', ''
    else:
        return winner_rank, loser_rank

# apply function to create new columns for home and away team ranks
games_df['home_rank'], games_df['away_rank'] = zip(*games_df.apply(lambda row: determine_home_away_rank(row['loc_ind'], row['winner_rank'], row['loser_rank']), axis=1))

# # drop the values from winner rank and loser rank columns if they are not nuetral
# games_df.loc[games_df['loc_ind'] != 'N', 'winner_rank'] = ''
# games_df.loc[games_df['loc_ind'] != 'N', 'loser_rank'] = ''
    
# Apply function to create new columns for home and away team scores
games_df['home_team_score'], games_df['away_team_score'] = zip(*games_df.apply(lambda row: determine_home_away_score(row['loc_ind'], row['winner_pts'], row['loser_pts']), axis=1))


# create columns for neutral site games
# if the loc_ind column is N then the winner and loser columns are the neutral site winner and loser
games_df['neutral_winner'] = games_df.apply(lambda row: row['winner'] if row['loc_ind'] == 'N' else '', axis=1)
games_df['neutral_loser'] = games_df.apply(lambda row: row['loser'] if row['loc_ind'] == 'N' else '', axis=1)
# clear the @ out of the loc_ind column
games_df.loc[games_df['loc_ind'] == '@', 'loc_ind'] = ''
# Drop the loc_ind column
# games_df = games_df.drop(columns=['loc_ind'])

# create column to identify all post season games
games_df['post_season_game'] = games_df['playoff_game'] + games_df['conf_champ_game'] + games_df['bowl_game']
# change the values to 1 if the game is a post season game
games_df.loc[games_df['post_season_game'] != '', 'post_season_game'] = 'yes'
# 







In [10]:
# df.info()
# reverse the chronological order of the dataframe
# games_df = games_df.iloc[::-1]
# games_df.info()

In [11]:
def compute_statistics_optimized(df):
    # Preprocessing: create columns for each team's points regardless of home/away or winner/loser
    df['team1'] = df.apply(lambda x: x['winner'] if x['loc_ind'] == 'N' else x['home_team'], axis=1)
    df['team2'] = df.apply(lambda x: x['loser'] if x['loc_ind'] == 'N' else x['away_team'], axis=1)
    df['team1_pts'] = df.apply(lambda x: x['winner_pts'] if x['loc_ind'] == 'N' else x['home_team_score'], axis=1)
    df['team2_pts'] = df.apply(lambda x: x['loser_pts'] if x['loc_ind'] == 'N' else x['away_team_score'], axis=1)

    # Initialize statistics dataframe
    columns = ['season', 'team', 'wins', 'losses', 'ties', 'avg_points_scored', 'avg_points_conceded']
    stats_df = pd.DataFrame(columns=columns)

    # Combine data for both teams into a single dataframe for easier processing
    team1_df = df[['season', 'team1', 'team1_pts', 'team2_pts']].rename(columns={'team1': 'team', 'team1_pts': 'scored', 'team2_pts': 'conceded'})
    team2_df = df[['season', 'team2', 'team2_pts', 'team1_pts']].rename(columns={'team2': 'team', 'team2_pts': 'scored', 'team1_pts': 'conceded'})
    combined = pd.concat([team1_df, team2_df], ignore_index=True)

    # Calculate wins, losses, ties and Win %
    combined['wins'] = (combined['scored'] > combined['conceded']).astype(int)
    combined['losses'] = (combined['scored'] < combined['conceded']).astype(int)
    combined['ties'] = (combined['scored'] == combined['conceded']).astype(int)
    combined['win_pct'] = combined['wins'] / (combined['wins'] + combined['losses'] + combined['ties'])
    # Round to 3 decimal places
    combined['win_pct'] = combined['win_pct'].round(3)

    # Group by season and team and aggregate results
    stats_df = combined.groupby(['season', 'team']).agg(
        wins=('wins', 'sum'),
        losses=('losses', 'sum'),
        ties=('ties', 'sum'),
        avg_points_scored=('scored', 'mean'),
        avg_points_conceded=('conceded', 'mean')
    ).reset_index()

    return stats_df

# Compute the statistics using the optimized approach
team_stats_optimized = compute_statistics_optimized(games_df)
team_stats_optimized.tail()
df = team_stats_optimized


In [12]:
# Single block of code to calculate home/away records and average points scored and allowed

def calculate_team_metrics_v2(df):
    # Home Games DataFrame
    home_games = df[df['home_team'].notna()].copy()
    home_games['home_win'] = (home_games['home_team'] == home_games['winner']).astype(int)
    home_games['home_tie'] = (home_games['winner_pts'] == home_games['loser_pts']).astype(int)
    home_games['home_loss'] = 1 - home_games['home_win'] - home_games['home_tie']

    # Away Games DataFrame
    away_games = df[df['away_team'].notna()].copy()
    away_games['away_win'] = (away_games['away_team'] == away_games['winner']).astype(int)
    away_games['away_tie'] = (away_games['winner_pts'] == away_games['loser_pts']).astype(int)
    away_games['away_loss'] = 1 - away_games['away_win'] - away_games['away_tie']

    # Grouping by season and team for home and away records
    home_record = home_games.groupby(['season', 'home_team']).agg(
        home_wins=('home_win', 'sum'),
        home_ties=('home_tie', 'sum'),
        home_losses=('home_loss', 'sum'),
        home_games=('home_win', 'count')
    ).reset_index()

    away_record = away_games.groupby(['season', 'away_team']).agg(
        away_wins=('away_win', 'sum'),
        away_ties=('away_tie', 'sum'),
        away_losses=('away_loss', 'sum'),
        away_games=('away_win', 'count')
    ).reset_index()

    # Grouping by season and team for home and away points
    home_points = home_games.groupby(['season', 'home_team']).agg(
        home_points_scored=('winner_pts', 'sum'),
        home_points_allowed=('loser_pts', 'sum')
    ).reset_index()

    away_points = away_games.groupby(['season', 'away_team']).agg(
        away_points_scored=('winner_pts', 'sum'),
        away_points_allowed=('loser_pts', 'sum')
    ).reset_index()

    # Merging records and points DataFrames
    merged_record = pd.merge(home_record, away_record, left_on=['season', 'home_team'], right_on=['season', 'away_team'], how='outer')
    merged_record['team'] = merged_record['home_team'].combine_first(merged_record['away_team'])

    merged_points = pd.merge(home_points, away_points, left_on=['season', 'home_team'], right_on=['season', 'away_team'], how='outer')
    merged_points['team'] = merged_points['home_team'].combine_first(merged_points['away_team'])

    merged_record.drop(['home_team', 'away_team'], axis=1, inplace=True)
    merged_points.drop(['home_team', 'away_team'], axis=1, inplace=True)

    final_df = pd.merge(merged_record, merged_points, on=['season', 'team'], how='outer')

    # Calculating average points
    final_df['avg_home_points_scored'] = final_df['home_points_scored'] / final_df['home_games']
    final_df['avg_home_points_allowed'] = final_df['home_points_allowed'] / final_df['home_games']
    final_df['avg_away_points_scored'] = final_df['away_points_scored'] / final_df['away_games']
    final_df['avg_away_points_allowed'] = final_df['away_points_allowed'] / final_df['away_games']

    # Reordering the columns for better readability
    reordered_columns = [
        'season', 'team', 
        'home_wins', 'home_losses', 'home_ties', 'home_games',
        'away_wins', 'away_losses', 'away_ties', 'away_games',
        'home_points_scored', 'home_points_allowed',
        'away_points_scored', 'away_points_allowed',
        'avg_home_points_scored', 'avg_home_points_allowed',
        'avg_away_points_scored', 'avg_away_points_allowed'
    ]
    final_df = final_df[reordered_columns]
    
    return final_df

# Run the refactored function on the existing DataFrame
home_away_df = calculate_team_metrics_v2(games_df)
home_away_df.tail()



,season,team,home_wins,home_losses,home_ties,home_games,away_wins,away_losses,away_ties,away_games,home_points_scored,home_points_allowed,away_points_scored,away_points_allowed,avg_home_points_scored,avg_home_points_allowed,avg_away_points_scored,avg_away_points_allowed
24257,2022,Western Carolina,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0,NaN,NaN,35.0,17.0,NaN,NaN,35.0,17.0
24258,2022,Western Illinois,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0,NaN,NaN,62.0,10.0,NaN,NaN,62.0,10.0
24259,2022,William & Mary,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,NaN,NaN,41.0,24.0,NaN,NaN,41.0,24.0
24260,2022,Wofford,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0,NaN,NaN,27.0,7.0,NaN,NaN,27.0,7.0
24261,2022,Youngstown State,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0,NaN,NaN,31.0,0.0,NaN,NaN,31.0,0.0


In [13]:
### TEMP CSV FOR CHECK
# Reverse the chronological order of the dataframe
games_df = games_df.iloc[::-1]

# # reduce the games_df to only include the years 1992 to the present
# games_df = games_df[games_df['season'] >= 2001]
# games_df.to_csv('../TEMP/check_partaial.csv', index=False)

In [14]:
# games_df.columns

In [15]:

df = games_df
## create rules for new dataframes slices
# 1. All games
# 2. Conference games - tagged in game type column
conf_df = df[df['game_type'] == 'Conference']
# 3. Non-Conference games - tagged in game type column
non_conf_df = df[df['game_type'] == 'Non-Conference']
# 4. All-Postseason Games - postseason column not empty
post_season_df = df[df['post_season_game'] != '']
# 5. Regular season games
reg_season_df = df[df['post_season_game'] == '']
# 6. Close Games - games with a margin of victory of 7 or less - tagged in margin column
close_games_df = df[df['margin'] == 'Close']
# 7. Blowout Games - games with a margin of victory of 21 or more - tagged in margin column
blowout_games_df = df[df['margin'] == 'Blowout']
# Neutral site games - neutral_winner column not empty
neutral_site_df = df[df['neutral_winner'] != '']

In [16]:
# away_games_df.columns

In [17]:
# LOOP THROUGH ALL SITUATIONAL DATAFRAMES AND AGGRIGATE IT ALL INTO A SINGLE TABLE
# Create a dictionary of dataframes and their associated prefixes
dfs_dict = {
    'all': games_df,
    'conf': conf_df,
    'non_conf': non_conf_df,
    'post_season': post_season_df,
    'reg_season': reg_season_df,
    'close_games': close_games_df,
    'blowout_games':blowout_games_df,
    'neutral_site': neutral_site_df,
    # 'campus_site': final_df_v2,
    # 'home_games': home_games_df,
    # 'away_games': away_games_df
}

# Process each dataframe and store the results in a dictionary
results_dict = {}
for prefix, df_slice in dfs_dict.items():
    results_dict[prefix] = compute_statistics_optimized(df_slice)

# Join all the results into a single dataframe
final_df = results_dict['all'].copy()
final_df.columns = ['season', 'team'] + [f'all_{col}' for col in final_df.columns if col not in ['season', 'team']]

# Merge the other dataframes using the season and team columns as keys
for prefix, result_df in results_dict.items():
    if prefix != 'all':
        new_cols = [f'{prefix}_{col}' for col in result_df.columns if col not in ['season', 'team']]
        result_df.columns = ['season', 'team'] + new_cols
        final_df = pd.merge(final_df, result_df, on=['season', 'team'], how='outer')

# Display a sample of the final dataframe
# final_df.head()


C:\Users\Justin\AppData\Local\Temp\ipykernel_2484\3946357835.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['team1'] = df.apply(lambda x: x['winner'] if x['loc_ind'] == 'N' else x['home_team'], axis=1)
C:\Users\Justin\AppData\Local\Temp\ipykernel_2484\3946357835.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['team2'] = df.apply(lambda x: x['loser'] if x['loc_ind'] == 'N' else x['away_team'], axis=1)
C:\Users\Justin\AppData\Local\Temp\ipykernel_2484\3946357835.py:5: SettingWithCopyWarning: 
A

In [18]:
## add the data from the home and away dataframe to the final_df
# Merge the other dataframes using the season and team columns as keys

# final_df = pd.merge(final_df, home_away_df, on=['season', 'team'], how='outer')


# # calculate win percentages for home and away games
# home_away_df['home_win_pct'] = home_away_df['home_wins'] / home_away_df['home_games']
# home_away_df['away_win_pct'] = home_away_df['away_wins'] / home_away_df['away_games']
# # home_away_df.info()





In [19]:
# Use the season and team columns as keys to merge the dataframes and add the home/away data to new columns
final_df = pd.merge(final_df, home_away_df, on=['season', 'team'], how='outer')

# do the win percentage calculation for all of the catagories in the final_df - all, conf, non_conf, post_season, reg_season, close_games, blowout_games, neutral_site
# calculate win percentages for all games
final_df['all_win_pct'] = final_df['all_wins'] / (final_df['all_wins'] + final_df['all_losses'] + final_df['all_ties'])
# calculate win percentages for conference games
final_df['conf_win_pct'] = final_df['conf_wins'] / (final_df['conf_wins'] + final_df['conf_losses'] + final_df['conf_ties'])
# calculate win percentages for non-conference games
final_df['non_conf_win_pct'] = final_df['non_conf_wins'] / (final_df['non_conf_wins'] + final_df['non_conf_losses'] + final_df['non_conf_ties'])
# calculate win percentages for post-season games
final_df['post_season_win_pct'] = final_df['post_season_wins'] / (final_df['post_season_wins'] + final_df['post_season_losses'] + final_df['post_season_ties'])
# calculate win percentages for regular season games
final_df['reg_season_win_pct'] = final_df['reg_season_wins'] / (final_df['reg_season_wins'] + final_df['reg_season_losses'] + final_df['reg_season_ties'])
# calculate win percentages for close games
final_df['close_games_win_pct'] = final_df['close_games_wins'] / (final_df['close_games_wins'] + final_df['close_games_losses'] + final_df['close_games_ties'])
# calculate win percentages for blowout games
final_df['blowout_games_win_pct'] = final_df['blowout_games_wins'] / (final_df['blowout_games_wins'] + final_df['blowout_games_losses'])
# calculate win percentages for neutral site games
final_df['neutral_site_win_pct'] = final_df['neutral_site_wins'] / (final_df['neutral_site_wins'] + final_df['neutral_site_losses'] + final_df['neutral_site_ties'])


# Display a sample of the final dataframe
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24461 entries, 0 to 24460
Data columns (total 66 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   season                             24461 non-null  int64  
 1   team                               24461 non-null  object 
 2   all_wins                           24314 non-null  float64
 3   all_losses                         24314 non-null  float64
 4   all_ties                           24314 non-null  float64
 5   all_avg_points_scored              24313 non-null  float64
 6   all_avg_points_conceded            24313 non-null  float64
 7   conf_wins                          7301 non-null   float64
 8   conf_losses                        7301 non-null   float64
 9   conf_ties                          7301 non-null   float64
 10  conf_avg_points_scored             7301 non-null   float64
 11  conf_avg_points_conceded           7301 non-null   flo

In [20]:
## Add conference affiliation to teams
import ast # for literal_eval function

conference_df = pd.read_csv('../data/yearly_conference_members.csv')

# rename Unnamed: 0 to year in conference_df
conference_df = conference_df.rename(columns={'Unnamed: 0': 'Year'})

# Convert string representations of lists back into actual lists
for col in conference_df.columns[1:]:
    conference_df[col] = conference_df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


# Function to retrieve the conference based on year and team name
def get_conference(year, team_name, conference_lookup):
    """
    Given a year and team name, retrieve the conference the team belongs to.
    If no conference found, return "Independent/Unknown".
    """
    return conference_lookup.get((year, team_name), "Independent/Unknown")

# Create a lookup dictionary for team-year to conference mapping
conference_lookup = {}

for _, row in conference_df.iterrows():
    year = row['Year']
    for col, teams in row[1:].items():
        for team in teams:
            conference_lookup[(year, team)] = col

## Add conference affiliation to teams
final_df['conference'] = final_df.apply(lambda row: get_conference(row['season'], row['team'], conference_lookup), axis=1)


In [21]:
final_df.columns

# Change all win lose and tie columns to integers
# create list of columns to change

# list of columns that should be integers
# all coulmns with win, lose, tie in the name should be integers
int_cols = ['all_wins', 'all_losses', 'all_ties', 
            'conf_wins', 'conf_losses', 'conf_ties', 
            'non_conf_wins', 'non_conf_losses', 'non_conf_ties', 
            'post_season_wins', 'post_season_losses', 'post_season_ties', 
            'reg_season_wins', 'reg_season_losses', 'reg_season_ties', 
            'neutral_site_wins', 'neutral_site_losses', 'neutral_site_ties',
            'close_games_wins', 'close_games_losses', 
            'blowout_games_wins', 'blowout_games_losses']

# fill and null values with 0
final_df[int_cols] = final_df[int_cols].fillna(0)
            

final_df['all_wins'] = final_df['all_wins'].astype(int)
final_df['all_losses'] = final_df['all_losses'].astype(int)
final_df['all_ties'] = final_df['all_ties'].astype(int)
final_df['conf_wins'] = final_df['conf_wins'].astype(int)
final_df['conf_losses'] = final_df['conf_losses'].astype(int)
final_df['conf_ties'] = final_df['conf_ties'].astype(int)
final_df['non_conf_wins'] = final_df['non_conf_wins'].astype(int)
final_df['non_conf_losses'] = final_df['non_conf_losses'].astype(int)
final_df['non_conf_ties'] = final_df['non_conf_ties'].astype(int)
final_df['post_season_wins'] = final_df['post_season_wins'].astype(int)
final_df['post_season_losses'] = final_df['post_season_losses'].astype(int)
final_df['post_season_ties'] = final_df['post_season_ties'].astype(int)
final_df['reg_season_wins'] = final_df['reg_season_wins'].astype(int)
final_df['reg_season_losses'] = final_df['reg_season_losses'].astype(int)
final_df['reg_season_ties'] = final_df['reg_season_ties'].astype(int)
final_df['close_games_wins'] = final_df['close_games_wins'].astype(int)
final_df['close_games_losses'] = final_df['close_games_losses'].astype(int)
final_df['blowout_games_wins'] = final_df['blowout_games_wins'].astype(int)
final_df['blowout_games_losses'] = final_df['blowout_games_losses'].astype(int)
final_df['neutral_site_wins'] = final_df['neutral_site_wins'].astype(int)
final_df['neutral_site_losses'] = final_df['neutral_site_losses'].astype(int)
final_df['neutral_site_ties'] = final_df['neutral_site_ties'].astype(int)


In [22]:

final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24461 entries, 0 to 24460
Data columns (total 67 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   season                             24461 non-null  int64  
 1   team                               24461 non-null  object 
 2   all_wins                           24461 non-null  int32  
 3   all_losses                         24461 non-null  int32  
 4   all_ties                           24461 non-null  int32  
 5   all_avg_points_scored              24313 non-null  float64
 6   all_avg_points_conceded            24313 non-null  float64
 7   conf_wins                          24461 non-null  int32  
 8   conf_losses                        24461 non-null  int32  
 9   conf_ties                          24461 non-null  int32  
 10  conf_avg_points_scored             7301 non-null   float64
 11  conf_avg_points_conceded           7301 non-null   flo

In [23]:
## OUTPUT THE FINAL DATAFRAME TO CSV
final_df.to_csv('../data/viz_table/year_by_year_team_stats_with_splits.csv', index=False)

In [24]:
## Print the length of each dataframe with a description
print('All Games: ', len(df))
print('Conference Games: ', len(conf_df))
print('Non-Conference Games: ', len(non_conf_df))
print('All Postseason Games: ', len(post_season_df))
print('Regular Season Games: ', len(reg_season_df))
print('Close Games: ', len(close_games_df))
print('Blowout Games: ', len(blowout_games_df))
print('Neutral Site Games: ', len(neutral_site_df))
print('Campus Site Games: ', len(campus_site_df))




All Games:  77718
Conference Games:  21016
Non-Conference Games:  56702
All Postseason Games:  1848
Regular Season Games:  75870
Close Games:  19568
Blowout Games:  25557
Neutral Site Games:  3622


NameError: name 'campus_site_df' is not defined

In [ ]:
## Code to create the split statistics for each team and year
# Splits - Overall, Regular Season, home, Away, Nuetral, Conference_Championship, Bowl, Playoff, games while ranked, games vs ranked opponents, games as ranked team



In [ ]:
## Chat GPT - Attempt


In [ ]:
# final_df.head(30)

final_df.columns


In [ ]:

# save to csv
final_df.to_csv('../data/cfb_yearly_summary_stats.csv', index=False)